In [2]:
#bring in relevant libraries
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import numpy as np
import re
import requests
import datetime
from datetime import datetime, timedelta
import time
import httplib, urllib, json, locale
from urlparse import urlparse
import operator
import bson
from collections import OrderedDict
from pymongo import MongoClient
from bson.objectid import ObjectId
from collections import Counter
from collections import defaultdict
from itertools import chain
from collections import defaultdict
import itertools

In [89]:
merchant_ids = []
#connecting to our db
connection = MongoClient(host = '')
db=
merchant = db['Merchants']
items = db['Items']
for merchants in merchant.find():
     merchant_ids.append(str(merchants['_id']))
for MERCHANT_ID in merchant_ids:
    locale.setlocale(locale.LC_ALL, 'en_US')
    doc1 = []
    for doc in merchant.find({'_id':ObjectId(MERCHANT_ID)}):
        doc1.append(doc)
    try:
        access_token = doc1[0]['pos_access_token']
        location_id = doc1[0]['location_ids'][0]        
    except:
        continue
    request_headers = {'Authorization': 'Bearer ' + access_token,
                           'Accept': 'application/json',
                           'Content-Type': 'application/json'}    
    connection = httplib.HTTPSConnection('connect.squareup.com')

    def format_money(amount):
        return locale.currency(amount / 100.)


        # Obtains all of the business's location IDs. Each location has its own collection of payments.
    def get_location_ids():
        request_path = '/v1/me/locations'
        connection.request('GET', request_path, '', request_headers)
        response = connection.getresponse()

        # Transform the JSON array of locations into a Python list
        locations = json.loads(response.read())

        location_ids = []
        for location in locations:
            if location is None:
                continue
        location_ids.append(location['id'])

        return location_ids    

    def get_payments_month(location_ids):

        #get time right now, timezone should be correct if computer is synced properly
        d = datetime.utcnow()
        diff_time = datetime.utcnow() - timedelta(days = 1)


          #time_now
          # Make sure to URL-encode all parameters

        #turn current time into a bunch of strings that we can add together to form link parameters below
        year,month,day,hour,minute,second = str(d.year),str(d.month),str(d.day),str(d.hour),str(d.minute),str(d.second)
        year_diff,month_diff,day_diff,hour_diff,minute_diff,second_diff = str(diff_time.year),str(diff_time.month),str(diff_time.day),str(diff_time.hour),str(diff_time.minute),str(diff_time.second)

        #if any of our dates/times are only one digit, we add a '0' to the front to make them suitable for URL 
        if len(day) == 1:
            day = '0'+ day
        if len(month) == 1:
            month = '0'+ month
        if len(hour) == 1:
            hour = '0' + hour
        if len(minute) == 1:
            minute = '0' + minute
        if len(second) == 1:
            second = '0' + second

        if len(day_diff) == 1:
            day_diff = '0'+ day_diff
        if len(month_diff) == 1:
            month_diff = '0'+ month_diff
        if len(hour_diff)==1:
            hour_diff = '0' + hour_diff

        parameters = urllib.urlencode({'begin_time': year_diff + '-' + month_diff + '-' + day_diff + 'T'+ hour_diff + ':00:00',
                                         'end_time': year + '-' + month + '-' + day + 'T' + hour + ':00:00'})
        payments = []

          # For each location...
        for location_id in location_ids:

            print 'Downloading payments for location with ID ' + location_id + '...'

            request_path = '/v1/' + location_id + '/payments?' + parameters
            more_results = True

            # ...as long as there are more payments to download from the location...
            while more_results:

              # ...send a GET request to /v1/LOCATION_ID/payments
                connection.request('GET', request_path, '', request_headers)
                response = connection.getresponse()

              # Read the response body JSON into the cumulative list of results
                payments = payments + json.loads(response.read())

              # Check whether pagination information is included in a response header, indicating more results
                pagination_header = response.getheader('link', '')
                if "rel='next'" not in pagination_header:
                    more_results = False
                else:

                # Extract the next batch URL from the header.
                #
                # Pagination headers have the following format:
                # <https://connect.squareup.com/v1/LOCATION_ID/payments?batch_token=BATCH_TOKEN>;rel='next'
                # This line extracts the URL from the angle brackets surrounding it.
                    next_batch_url = urlparse(pagination_header.split('<')[1].split('>')[0])

                    request_path = next_batch_url.path + '?' + next_batch_url.query

          # Remove potential duplicate values from the list of payments
        seen_payment_ids = set()
        unique_payments = []

        for payment in payments:
            try:
                if payment['id'] in seen_payment_ids:
                    coprice_ntinue
                seen_payment_ids.add(payment['id'])
                unique_payments.append(payment)
            except:
                continue
        return unique_payments


    if __name__ == '__main__':
        # Get all 2015 payments from all of the business's locations

        payments = get_payments_month(get_location_ids())
        pay_data_month = payments
        if len(pay_data_month) == 0:
            continue
        # Print a sales summary report of the payments
        #print_sales_report(payments)
    try:
        #connecting to the items df, this will allow us to bring in price and category id
        connection = httplib.HTTPSConnection('connect.squareup.com')
        request_path = '/v1/'+location_id+'/items'
        connection.request('GET', request_path, '', request_headers)
        response = connection.getresponse()
        # Convert the returned JSON body into an array of locations you can work with.
        items_data = json.loads(response.read())

        #initialize empty lists that will be populated with data from our JSON api file
        cat_name = []
        cat_id = []
        item_name = []
        price_list = []

        for q in items_data:
            try:
                cat_name.append(str(q['category']['name']).lower())
            except:
                cat_name.append('')
            try:
                cat_id.append(q['category_id'])
            except:
                cat_id.append('')
            try:
                item_name.append(q['name'])
            except:
                item_name.append('')
            try:
                price_list.append(float(q['variations'][0]['price_money']['amount'])/100)
            except:
                price_list.append(np.nan)

        #make a dataframe from the lists
        df2 = pd.DataFrame({'category_id':cat_id,'cat_name':cat_name, 'name':item_name, 'price':price_list})
        #group by to get average price for each category
        avg_price_df = df2.groupby(['category_id','cat_name']).mean()
        #convert our dataframe to a dictionary that has the important inifo in a {'cat_name':[price,cat_id]} format
        price_id_dict = avg_price_df.reset_index().set_index('cat_name').to_dict('index')
        
        #make empty list of various grouped categories
        categories_together = []
        #for every entry in our payments data, we pull the category name by iterating through the itemizations
        for q in pay_data_month:
            cat_name_mon = []
            for i in q['itemizations']:
                cat_name_mon.append(str(i['item_detail']['category_name']).lower())
            #append each category into a list from the transaction, then add this list to a greater list
            categories_together.append(cat_name_mon)


        #for each sublist, we get rid of blank entries, make a set out of them (remove duplicates), and ensure that only orders
        #of length 2 or greater are presented. Then we make a pairing of lists that contain each of our categories.
        fin_list = []
        for sublist in categories_together:
            sublist = filter(lambda a: a != '', sublist)
            search_set = set(sublist)
            for search in search_set:
                if len(sublist)>1:
                    for i in range(len(sublist)):
                        #sublist.remove('')
                        #if sublist[i]=='':
                         #   del sublist[i]
                        if sublist[i] == search:
                            fin_list.append(sublist)
                            break
        #make a dataframe out of all of the lists
        orders = pd.DataFrame(fin_list)

        #extract all of the categories into a list
        categories = list(set(cat_name))

        for item in categories:     #######figure out if we need this
            item_list = []
            item_list.append(item)
            for columns in orders:
                orders = orders[~orders[columns].isin(item_list)]
                
                
        #replace all of the items in the dataframe that are the same as the main category so as to not duplicate
        for item in categories:    
            orders.replace({item:None})
        #for every category, make a list of lists containing the orders that contain it
        df_list = []
        for item in categories:
            item_specific_list = []
            for l in fin_list:
                if item in l:
                    item_specific_list.append(l)
            df_list.append(item_specific_list)

        #for every order, we set the category as the index and then make a list of dataframes with each category dataframe
        specific_dfs = []
        for i,item in enumerate(categories):
            lis = df_list[i]
            for orders in lis:
                if item in orders:
                    dataframe = pd.DataFrame(lis)
                    dataframe['category'] = item
                    dataframe = dataframe.set_index('category', drop = True)
                    col_list = list(dataframe)
            specific_dfs.append(dataframe)

        #replace all of the items in the dataframe that are the same as the main category so as to not duplicate
        for i,item in enumerate(categories):    
            specific_dfs[i] = specific_dfs[i].replace({item:None})

#for i, df in enumerate(specific_dfs):
        main_dict ={}
        for i,df in enumerate(specific_dfs):
            dictionary = {}
            temp_cat = df.index[0]
            dictionary[temp_cat] = df.apply(pd.value_counts).sum(axis = 1).to_dict()
            main_dict.update(dictionary)
        #print index_cat
        
        
        #we are essentially merging the price/category dictionary with the item/quantity dictionary
        dict3 = defaultdict(list)
        for k, v in chain(main_dict.items(), price_id_dict.items()):
            dict3[k].append(v)
        
        dict3 = dict(dict3)
        list_of_associations = []
        association_list = []

        
        #remove all the blank entries from every list of orders, and also remove orders that were just a single item
        for sublist in categories_together:
                    sublist = filter(lambda a: a != '', sublist)
                    search_set = set(sublist)
                    for search in search_set:
                        if len(sublist)>1:
                            for i in range(len(sublist)):
                                if sublist[i] == search:
                                    fin_list.append(sublist)
        #make tuples of categorires and lists of orders that include those categories
        for word in categories:
            association_list = []
            for lists in fin_list:
                for i in range(len(lists)):
                    if lists[i] == word:
                        association_list.append(lists)
                        break
            word_tuple = (word, association_list)
            list_of_associations.append(word_tuple)


        #really tricky section, we convert all our order lists to tuples, count the tuples, then turn the Counter object into a dictionary and
        #add to a list
        list_with_counts = []
        for word_tup in list_of_associations:
            counts_list = []
            counted = dict(Counter(tuple(e) for e in word_tup[1]))
            for i,lists in enumerate(word_tup[1]):
                counts_list.append(lists)
            list_with_counts.append([word_tup[0],counted])
            
        dict_of_associations = dict(list_of_associations)
        dict_of_counts = dict(list_with_counts)
        
        upload_dict = {}
        
        for keys,values in dict_of_counts.iteritems():
            cats_list = []
            for dicts,count in values.iteritems():
                for cats in dicts:
                    if keys != cats:
                        cats_list.append(cats)
                        cats_list = list(set(cats_list))
            upload_dict[keys] = [cats_list,count]
            
        upload_dict_df = pd.DataFrame(upload_dict).transpose()
        upload_dict_df.columns = ['order combos','combos_count']
        upload_dict = upload_dict_df.to_dict('index')

        #post to Items db
        items.update_many({
                          'merchant_uid': MERCHANT_ID
                        },{
                          '$set': {
                              'order_quantity_info':upload_dict,
                              'category_info':dict3

                            }

                          }
                        , upsert=True)
    except:
        continue

0
{'charcuterie': {'charcuterie': 5.0, 'grocery': 6.0, 'wine': 3.0, 'bread': 2.0}, '': {'price': 12.975, 'category_id': ''}, 'cheese': {'price': 21.990000000000006, 'category_id': u'EEDBD7E8-F5CC-4AF0-8366-E2E3A6F85207'}, 'catering': [{'grocery': 3.0, 'catering': 3.0, 'bread': 3.0}, {'price': 55.39300000000001, 'category_id': u'KKKLIQ2O7OHTIHHAWY3BHB5Y'}], 'happy hour': [{'grocery': 2.0, 'wine': 2.0, 'sandwiches': 6.0, 'drinks': 6.0}, {'price': 18.331666666666667, 'category_id': u'JRWLHZCLQVRB22JSROEMXSFG'}], 'breakfast': [{}, {'price': 3.5081250000000006, 'category_id': u'94B075F2-F7BD-4793-8006-B2740AB58C86'}], 'salads': [{'grocery': 2.0, 'wine': 2.0, 'sandwiches': 2.0, 'grilled cheeses': 2.0, 'drinks': 4.0}, {'price': 10.49, 'category_id': u'3592C730-C045-4A8F-8ACB-CD7DB8665399'}], 'sides': [{'grocery': 6.0, 'sandwiches': 89.0, 'grilled cheeses': 14.0, 'drinks': 6.0}, {'price': 2.25, 'category_id': u'AE240C9C-5D54-4FFB-B8AA-4D133BBEFF35'}], 'drinks': [{'happy hour': 3.0, 'salads': 4